In [3]:
!pip install torch torchvision transformers pillow gradio --upgrade

  Using cached pillow-11.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.1 kB)


In [4]:
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import gradio as gr


device = "cuda" if torch.cuda.is_available() else "cpu"


print("Loading the VQA model and processor...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)
print("VQA model and processor loaded successfully.")


def answer_question(image, question):
    try:

        image = image.resize((480, 480))


        detailed_question = f"Please provide a detailed answer: {question}"


        inputs = processor(images=image, text=detailed_question, return_tensors="pt").to(device)


        with torch.no_grad():
            out = model.generate(**inputs, max_length=100, num_beams=5, early_stopping=True)


        answer = processor.decode(out[0], skip_special_tokens=True)


        detailed_answer = f"The model's answer is: '{answer}'. If you need more information, feel free to ask follow-up questions related to the image or clarify specific aspects you'd like to know about."

        return detailed_answer

    except Exception as e:
        return f"An error occurred: {str(e)}"

#  Gradio interface
title = "Visual Question Answering (VQA)"
description = "Upload an image and ask any question about it. The model will provide an appropriate answer based on the visual content of the image.Developed by Panchadip"

interface = gr.Interface(
    fn=answer_question,
    inputs=[gr.Image(type="pil"), gr.Textbox(lines=2, placeholder="Ask a question about the image...")],
    outputs="text",
    title=title,
    description=description,
    allow_flagging="never",
    live=True,
)

# Launching the interface
if __name__ == "__main__":
    interface.launch(share=True)


Loading the VQA model and processor...
VQA model and processor loaded successfully.


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bb638204f3dcff3b8d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
